### Import TensorFlow

In [1]:
!pip3 install tensorflow --user
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

from os import walk
import cv2

from sklearn.model_selection import train_test_split 
import numpy as np

You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
# (train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

def loadData(dirPath):
    X=[]
    y=[]
    for (dirpath, dirnames, filenames) in walk(dirPath):
        for filename in filenames:
            if filename != '.DS_Store': 
                img = cv2.cvtColor(cv2.imread(dirPath+'/'+filename), cv2.COLOR_BGR2GRAY)
                label = int(filename.split('_')[0])
                X.append(img)
                y.append(label)
    return X, y

X_nir, y_nir = loadData('dataset2/NIR')
X_vsb, y_vsb = loadData('dataset2/VSB')

X_train_nir, X_test_nir, y_train_nir, y_test_nir = train_test_split(X_nir, y_nir, test_size=0.2, random_state=42)
X_train_vsb, X_test_vsb, y_train_vsb, y_test_vsb = train_test_split(X_vsb, y_vsb, test_size=0.2, random_state=42)

# # Normalize pixel values to be between 0 and 1
# X_train_nir, X_test_nir = X_train_nir / 255.0, X_test_nir / 255.0
# X_train_vsb, X_test_vsb = X_train_vsb / 255.0, X_test_vsb / 255.0

In [3]:
# NIR
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(51))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 97, 97, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 135424)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                8

In [4]:
#VSB
model1 = models.Sequential()
model1.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 1)))
model1.add(layers.MaxPooling2D((2, 2)))
model1.add(layers.Conv2D(64, (3, 3), activation='relu'))
model1.add(layers.MaxPooling2D((2, 2)))
model1.add(layers.Conv2D(64, (3, 3), activation='relu'))
model1.add(layers.Flatten())
model1.add(layers.Dense(64, activation='relu'))
model1.add(layers.Dense(51))
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 198, 198, 32)      320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 99, 99, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 97, 97, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 46, 46, 64)        36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 135424)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)               

### Compile and train the model

In [5]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(np.array(X_train_nir).reshape(-1, 200, 200, 1), np.array(y_train_nir), epochs=20)

model1.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history1 = model1.fit(np.array(X_train_vsb).reshape(-1, 200, 200, 1), np.array(y_train_vsb), epochs=20)

Epoch 1/20
25/25 [==============================] - 18s 723ms/step - loss: 40.1088 - accuracy: 0.0338
Epoch 2/20
25/25 [==============================] - 20s 786ms/step - loss: 3.8393 - accuracy: 0.1338
Epoch 3/20
25/25 [==============================] - 18s 718ms/step - loss: 3.5234 - accuracy: 0.1825
Epoch 4/20
25/25 [==============================] - 17s 678ms/step - loss: 3.1755 - accuracy: 0.2600
Epoch 5/20
25/25 [==============================] - 17s 680ms/step - loss: 3.0037 - accuracy: 0.3075
Epoch 6/20
25/25 [==============================] - 17s 685ms/step - loss: 2.1123 - accuracy: 0.5288
Epoch 7/20
25/25 [==============================] - 19s 755ms/step - loss: 0.6521 - accuracy: 0.8475
Epoch 8/20
25/25 [==============================] - 18s 734ms/step - loss: 0.4401 - accuracy: 0.9362
Epoch 9/20
25/25 [==============================] - 18s 726ms/step - loss: 0.2568 - accuracy: 0.9588
Epoch 10/20
25/25 [==============================] - 17s 688ms/step - loss: 0.0854 - accur

### Evaluate the model

In [9]:
# plt.plot(history.history['accuracy'], label='accuracy')
# # plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.ylim([0.5, 1])
# plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(np.array(X_test_nir).reshape(-1, 200, 200, 1),  np.array(y_test_nir), verbose=2)
test_loss1, test_acc1 = model1.evaluate(np.array(X_test_vsb).reshape(-1, 200, 200, 1),  np.array(y_test_vsb), verbose=2)

7/7 - 1s - loss: 5.6325 - accuracy: 0.6250
7/7 - 1s - loss: 1.5999 - accuracy: 0.8300


In [10]:
print(test_acc)
print(test_acc1)

0.625
0.8299999833106995


Our simple CNN has achieved a test accuracy of over 70%. Not bad for a few lines of code! For another CNN style, see an example using the Keras subclassing API and a `tf.GradientTape` [here](https://www.tensorflow.org/tutorials/quickstart/advanced).